In [8]:
import numpy as np
import pandas as pd
import os
from covid_data import load_covid_data
import data_context_map.pattern_miner as pm

### Run COVID-19 Explainable AI to identify counties with unusually high / low COVID-19 Deaths per 100K

#### See https://svi.cdc.gov/Documents/Data/2014_SVI_Data/SVI2014Documentation.pdf for the social vulnerability data dictionary

In [9]:
!wget -N http://ptc-static-file-service.jhub:8080/covid_19_data.zip && unzip -o covid_19_data.zip && rm covid_19_data.zip

--2020-08-18 23:47:14--  http://ptc-static-file-service.jhub:8080/covid_19_data.zip
Resolving ptc-static-file-service.jhub (ptc-static-file-service.jhub)... 10.109.81.218
Connecting to ptc-static-file-service.jhub (ptc-static-file-service.jhub)|10.109.81.218|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14390119 (14M) [application/zip]
Saving to: ‘covid_19_data.zip’

covid_19_data.zip   100%[===================>]  13.72M  --.-KB/s    in 0.05s   

2020-08-18 23:47:14 (282 MB/s) - ‘covid_19_data.zip’ saved [14390119/14390119]

Archive:  covid_19_data.zip
  inflating: Covid-19Notebook_Data/cdcs-social-vulnerability-index-svi-2016-overall-svi-county-level.csv  
  inflating: Covid-19Notebook_Data/county_pm25.csv  
  inflating: Covid-19Notebook_Data/us-county-health-rankings-2020.csv  


In [10]:
path = os.getcwd()+'/Covid-19Notebook_Data/'
df_health_rank = pd.read_csv(path+'us-county-health-rankings-2020.csv')
df_pm = pd.read_csv(path+'county_pm25.csv')
df_esri = pd.read_csv(path+'cdcs-social-vulnerability-index-svi-2016-overall-svi-county-level.csv')

In [11]:
from datetime import date, timedelta
head_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'

# Check the past 5 days for the most recent data
for i in range(5):
    day = date.today() - timedelta(days=i)
    url = head_url + day.strftime("%m-%d-%Y") + ".csv"
    try:
        df_daily_cases = pd.read_csv(url)
        break
    except:
        pass
    

In [12]:
# Only look at us cases
df_us_daily_cases = df_daily_cases[df_daily_cases['Country_Region']=='US'].copy()

# rename columns for convenience
df_us_daily_cases.rename(columns={'Admin2': 'county', 'Province_State': 'state', 'FIPS': 'fips', 'Deaths': 'deaths', 'Confirmed': 'cases'}, inplace=True)

# make all countries start with their first death
df_us_daily_cases = df_us_daily_cases[df_us_daily_cases['deaths']!=0]

# average polution particulates
avg_pm25 = df_pm.groupby("fips").pm25.agg("mean")

maxDeaths = df_us_daily_cases.groupby(['county', 'state', 'fips']).deaths.agg('max')  # get cumulative number of deaths
pop = df_health_rank.groupby(['county', 'state', 'fips']).population_2.agg('max')  # total population
probDeath = (np.log(maxDeaths / pop)).to_frame().reset_index().rename(columns={0:'prDeath'})

df = pd.merge(probDeath, df_health_rank, how='left', left_on=['county', 'state', 'fips'], right_on = ['county', 'state', 'fips'])
df = pd.merge(df, df_esri.drop(['state'], axis=1), how='left', left_on=['county', 'fips'], right_on=['county', 'fips'])
df = pd.merge(df, avg_pm25, how='left', left_on=['fips'], right_on=['fips'])

# population per square mile
df['pop_density']=df['population_2'] / df['area_sqmi']

# raw number indicators
rawnum = [c for c in df.columns if 'num_' in c]

# convert raw numbers to incidence levels
df[[r+"_per_100k" for r in rawnum]] = df[rawnum].div(df['population_2'], axis=0) * 100000
df.drop(rawnum, axis=1, inplace=True)

# drop 95% confidence intervals 
drop = [c for c in df.columns if '95' in c] + ['population']
df.drop(drop, axis=1, inplace=True)

# remove raw counts in favor of percentages
drop = [c for c in df.columns if c[:2] == 'e_' or c[:2] == 'm_'] + ['state', 'st_abbr', 'st']

df.drop(drop, axis=1, inplace=True)

### Multi-variate Pattern Mining

In [13]:
out = pm.DataContextMap(df.drop(['fips'], axis=1), es_thresh=0.75, minsup=0.03, dependent='prDeath', holdout=2, max_pattern=300, show_causal=True, verbose=0)
out.render()

Output()

Output()

### Correlation Mining

In [14]:
ctable = pm.CorrelationTable(df.drop(['fips'], axis=1), 'prDeath', es_thresh=0.6, minsup=0.1, max_depth=3, max_pattern=1000, holdout=2, verbose=0)
ctable.render()

CorrelationTableWidget(data=[{'Variable 1': 'years_of_potential_life_lost_rate', 'Variable 2': 'prDeath', 'Pea…

### U.S. County Map View

In [15]:
usminer = pm.USCountyMiner(df, dependent='prDeath', es_thresh=0.7, minsup=0.05, max_depth=3, max_pattern=300, fips='fips')
usminer.render()

USCountyWidget(county_value={45001: 32.598508618230724, 22001: 144.71780028943556, 51001: 52.44970998395657, 1…